In [ ]:
!pip install transformers

In [4]:
import torch
from transformers.models.bert import BertModel, BertTokenizer


In [5]:
model_name = 'bert-base-uncased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [12]:
model = BertModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [14]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

# model output

In [16]:
test_input = 'this is a test sentence'
inputs = tokenizer(test_input, return_tensors='pt')

In [18]:
model.eval()
with torch.no_grad():
  output = model(**inputs)

In [20]:
output[0]

tensor([[[-0.1276,  0.1170, -0.0797,  ..., -0.4029,  0.1983,  0.6898],
         [-0.7166, -0.4734, -0.0419,  ..., -0.6309,  0.5737,  0.6857],
         [-0.3594, -0.4751,  0.4444,  ..., -0.3402,  0.1128,  1.0230],
         ...,
         [ 0.5048, -0.4322, -0.2758,  ..., -0.5477,  0.0312, -0.3804],
         [-0.2744, -0.1734, -0.4679,  ...,  0.3030,  0.0412, -0.1826],
         [ 0.6264,  0.0517, -0.5116,  ...,  0.2888, -0.8344, -0.2331]]])

In [22]:
output[1].shape

torch.Size([1, 768])

In [25]:
output[2][0]#embedding

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [-0.6485,  0.6739, -0.0932,  ...,  0.4475,  0.6696,  0.1820],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [ 0.6010, -0.6970, -0.2001,  ...,  0.2960,  0.2060, -1.7181],
         [ 0.8323,  0.2878,  0.0021,  ...,  0.2628, -1.1310, -1.2708],
         [-0.1481, -0.2948, -0.1690,  ..., -0.5009,  0.2544, -0.0700]]])

In [26]:
output[2][1]#first_layer

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]])

# add_norm

In [27]:
embeddings = output[2][0]

In [28]:
layer = model.encoder.layer[0]

In [29]:
layer

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [30]:
#这里是BertSelfAttention的输出，BertAttention包含BertSelfAttention和BertSelfOutput两个类，BertSelfAttention输出一个元组（context_layer, attention_probs）
#attention_probs为可选输出参数
mha_output = layer.attention.self(embeddings)

In [37]:
mha_output[0].shape

torch.Size([1, 7, 768])

In [40]:
att_output = layer.attention.output(mha_output[0],embeddings)

In [41]:
att_output#多头注意力和残差链接的输出

tensor([[[ 0.6143, -0.2061, -0.5682,  ...,  0.2557,  0.1224,  0.3388],
         [-0.6751,  1.0598, -0.2210,  ...,  0.6263,  1.1682,  0.1998],
         [-0.5969,  0.1463, -0.5549,  ...,  0.5272,  0.5071,  0.6439],
         ...,
         [ 1.3291, -1.1190, -0.1960,  ...,  0.6164,  0.4293, -2.4704],
         [ 1.8784,  0.5158,  0.1902,  ...,  0.5298, -1.3812, -2.0257],
         [ 0.7376, -0.7000, -0.0992,  ..., -0.9356,  0.5831, -0.4610]]],
       grad_fn=<NativeLayerNormBackward0>)

In [42]:
inter_output = layer.intermediate(att_output)

In [43]:
inter_output

tensor([[[-6.5263e-03, -4.5374e-02, -1.3537e-02,  ..., -3.3436e-02,
          -1.1251e-02, -1.3501e-03],
         [-7.4728e-02, -3.0861e-02, -6.9455e-04,  ..., -2.4787e-03,
          -3.7003e-02, -1.0936e-03],
         [-2.1356e-02, -7.4835e-04, -1.7961e-05,  ..., -4.2711e-04,
          -3.1277e-02, -6.5077e-03],
         ...,
         [-1.5633e-01, -1.5660e-02, -1.7595e-02,  ..., -1.8446e-03,
          -1.9851e-03, -3.5248e-02],
         [-1.6379e-01, -2.3173e-02, -1.2511e-03,  ..., -2.8440e-03,
          -1.5373e-01, -2.3979e-02],
         [-1.1530e-03, -5.6150e-02, -2.2487e-03,  ..., -4.7188e-03,
          -1.8783e-02, -1.3366e-05]]], grad_fn=<GeluBackward0>)

In [44]:
first_layer_encode_output = layer.output(inter_output,att_output)

In [45]:
first_layer_encode_output

tensor([[[ 0.1556, -0.0080, -0.0707,  ...,  0.0786,  0.0213,  0.0616],
         [-0.5333,  0.5799,  0.1044,  ...,  0.0241,  0.4888,  0.0161],
         [-1.0609, -0.3058, -0.5043,  ...,  0.1874,  0.2874,  0.4032],
         ...,
         [ 0.8206, -0.6656, -0.7054,  ...,  0.1347,  0.1117, -1.9040],
         [ 1.1128,  0.6603, -0.1509,  ...,  0.3253, -1.0006, -1.9106],
         [-0.0736,  0.0346,  0.0376,  ..., -0.4506,  0.6585, -0.0502]]],
       grad_fn=<NativeLayerNormBackward0>)

In [47]:
output[2][1] == first_layer_encode_output

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])